In [2]:
import kenlm
import os
import io
import nltk
import time
import csv
import re
import pandas as pd
import numpy as np
from nltk.tokenize import ToktokTokenizer
from string import punctuation
import matplotlib.pyplot as plt
import tarfile,os
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import spacy
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import PathLineSentences
import multiprocessing
from sklearn.neighbors import DistanceMetric
from operator import itemgetter 
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from gensim.models import FastText
from sklearn.naive_bayes import ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

# Importation des données

In [3]:
def measure_accuracy(pred, truth):
    good = 0
    len_testset = len(truth)

    for i in range(len_testset):
        if(truth[i] == pred[i]):
            good+=1
    return good/len_testset

In [4]:
train_set = pd.read_pickle("clean_trainset.pkl")
test_set = pd.read_pickle("clean_testset.pkl")

y_train = train_set["sign"]
y_test = test_set["sign"]

In [5]:
tfidf = TfidfVectorizer(max_features=4000)
X_train = tfidf.fit_transform(train_set["tokenized_text_string"].values).toarray()
X_test = tfidf.transform(test_set["tokenized_text_string"].values).toarray()

# Explore different models

In [155]:
model = DummyClassifier(strategy='most_frequent')
model.fit(X_train, y_train)
print(measure_accuracy(model.predict(X_test), y_test))

0.06477238007943782


In [109]:
gnb = ComplementNB()
model_nb = gnb.fit(X_train, y_train)
pred_NB = model_nb.predict(X_test)
print(measure_accuracy(pred_NB, y_test))

0.08457073021692636


In [46]:
model_clf = xgb.XGBClassifier(n_estimators=100, objective = 'multi:softmax')
model_clf = model_clf.fit(X_train, y_train)
pred_xgb = model_clf.predict(X_test)
measure_accuracy(pred_xgb, y_test)

0.08001833180568287

In [82]:
model_svm = svm.SVC()
model_svm = model_svm.fit(X_train, y_train)
preds_svm = model_svm.predict(X_test)
measure_accuracy(preds_svm, y_test)

0.08102658111824014

In [152]:
model_mlp = MLPClassifier(solver='lbfgs', alpha=1e-6, 
                     hidden_layer_sizes=(6, 5), learning_rate_init=0.001, random_state=1)

model_mlp = model_mlp.fit(X_train, y_train)
pred_mlp = model_mlp.predict(X_test)
measure_accuracy(pred_mlp, y_test)

C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.10171096853040025

In [153]:
print(classification_report(pred_mlp, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.01      0.05      0.01       264
           2       0.20      0.17      0.19      6912
           3       0.05      0.07      0.06      1499
           4       0.12      0.06      0.08      4461
           5       0.17      0.08      0.11      5921
           6       0.10      0.08      0.09      3371
           7       0.04      0.07      0.06      1298
           8       0.00      0.33      0.00         3
           9       0.01      0.03      0.02       742
          10       0.21      0.11      0.15      7125
          11       0.03      0.06      0.04      1134

    accuracy                           0.10     32730
   macro avg       0.08      0.09      0.07     32730
weighted avg       0.15      0.10      0.12     32730



C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
model_fc = RandomForestClassifier(max_depth=4, random_state=0)
model_fc = model_fc.fit(X_train, y_train)
pred_fc = model_fc.predict(X_test)

measure_accuracy(pred_fc, y_test)

0.07256339749465322

In [31]:
model_lr = LinearRegression()
model_lr = model_lr.fit(X_train, y_train)
pred_lr = model_lr.predict(X_test)

measure_accuracy(pred_lr, y_test)

0.0

In [32]:
model_lr = LogisticRegression(multi_class="multinomial", solver="newton-cg")
model_lr = model_lr.fit(X_train, y_train)
pred_lr = model_lr.predict(X_test)

measure_accuracy(pred_lr, y_test)

0.08026275588145432

In [18]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1, max_iter=200)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = ComplementNB()
clf4 = model_mlp = MLPClassifier(solver='lbfgs', alpha=1e-6,
                     hidden_layer_sizes=(6, 5), random_state=1, max_iter=300)

eclf1 = VotingClassifier(estimators=[
         ('lr', clf1), ('rf', clf2), ('gnb', clf3), ('mlp', clf4)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
pref_eclf = eclf1.predict(X_test)
measure_accuracy(pref_eclf, y_test)

C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.08912312862816987

# Best models

In [28]:
print(joblib.load('best_logreg.pkl').best_params_)
print(joblib.load('best_forest.pkl').best_params_)
print(joblib.load('best_NB.pkl').best_params_)

In [158]:
start = time.time()
clf1 = LogisticRegression(multi_class='multinomial', tol=0.0001, random_state=1, max_iter=200, C=1.0)
clf2 = RandomForestClassifier(n_estimators=100, random_state=1, min_samples_split=4)
clf3 = ComplementNB()
clf4 = MLPClassifier(solver='lbfgs', alpha=1e-6, learning_rate_init=0.05,
                     hidden_layer_sizes=(6, 6, 6, 6), random_state=1, max_iter=100)

eclf1 = VotingClassifier(estimators=[
         ('lr', clf1), ('rf', clf2), ('gnb', clf3), ('mlp', clf4)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
pref_eclf = eclf1.predict(X_test)

end = time.time()
print(end-start)

measure_accuracy(pref_eclf, y_test)

C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


213.74872660636902


0.09706691109074243

In [162]:
model_mlp = MLPClassifier(solver='lbfgs', alpha=1e-6, 
                     hidden_layer_sizes=(6, 5), learning_rate_init=0.001, random_state=1)

model_mlp = model_mlp.fit(X_train, y_train)
pred_mlp = model_mlp.predict(X_test)
measure_accuracy(pred_mlp, y_test)

C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.10171096853040025

# With Word2Vec

In [5]:
def word_average(sent, model):
    mean = []
    for word in sent:
        if word in model.wv.vocab:
            mean.append(model.wv.get_vector(word))

    if not mean:  # empty words
        # If a text is empty, return a vector of zeros.
        #logging.warning("cannot compute average owing to no vector for {}".format(sent))
        return np.zeros(len_embed)
    else:
        mean = np.array(mean).mean(axis=0)
        return mean


def word_average_list(row, model):
    average = word_average(row["tokenized_text"], model)
    return average

#model = Word2Vec.load("language_model/w2vmodel.model")
#len_embed = model.vector_size

#X_train = train_set.apply(lambda row: word_average_list(row, model), axis=1)
#X_test = test_set.apply(lambda row: word_average_list(row, model), axis=1)

# X_train.to_pickle("X_train_word2vec.pkl")
# X_test.to_pickle("X_test_word2vec.pkl")

In [119]:
X_train = np.vstack(pd.read_pickle("X_train_word2vec.pkl"))
X_test = np.vstack(pd.read_pickle("X_test_word2vec.pkl"))

In [122]:
model_mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1)

model_mlp = model_mlp.fit(np.vstack(X_train), y_train)
pred_mlp = model_mlp.predict(np.vstack(X_test))
measure_accuracy(pred_mlp, y_test)

0.06477238007943782

In [123]:
gnb = GaussianNB()
model_nb = gnb.fit(np.vstack(X_train), y_train)
pred_NB = model_nb.predict(np.vstack(X_test))
measure_accuracy(pred_NB, y_test)

0.0823709135349832

In [124]:
model_svm = svm.SVC()
model_svm = model_svm.fit(X_train, y_train)
preds_svm = model_svm.predict(X_test)
measure_accuracy(preds_svm, y_test)

0.06431408493736633

In [125]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
eclf1 = VotingClassifier(estimators=[
         ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
pref_eclf = eclf1.predict(X_test)
measure_accuracy(pref_eclf, y_test)

0.08991750687442714

# RandomSearch for MLP

In [92]:
model = MLPClassifier

#model_mlp = MLPClassifier(solver='lbfgs', alpha=1e-6,
#                     hidden_layer_sizes=(6, 5), random_state=1)


param = {
'vect__max_features': [1000, 2000, 3000, 4000],
'model__hidden_layer_sizes':  [(6,5), (5,5), (6,6), (7,5), (7,6), (4,5,2)],
'model__solver':  ['lbfgs', 'adam'],
'model__activation': ['logistic', 'relu', 'lbfgs'],
'model__learning_rate': ['constant','adaptive'],
'model__random_state': [1],
'model__alpha' : [1e-4, 1e-5, 1e-6]
}

    
vectorizer = TfidfVectorizer

pipe = Pipeline([
        ('vect', vectorizer()),
        ('model', model())
    ])
gs = RandomizedSearchCV(pipe,
                      param_distributions=param,  # what parameters values are we searching?
                      cv=10,
                      n_iter=20,
                      scoring='accuracy', verbose=10,
                      n_jobs=-1)  # 5-fold cross-validation.


# Fit GridSearch to training data.
gs.fit(train_set["tokenized_text_string"], y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 29.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 35.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 42.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 51.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 60.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 67.3min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 76.6min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 87.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 98.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 109.6min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 1

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                             ('model', MLPClassifier())]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'model__activation': ['logistic',
                                                              'relu', 'lbfgs'],
                                        'model__alpha': [0.0001, 1e-05, 1e-06],
                                        'model__hidden_layer_sizes': [(6, 5),
                                                                      (5, 5),
                                                                      (6, 6),
                                                                      (7, 5),
                                                                      (7, 6),
                                                                      (4, 5,
                                                                       2)],
       

# RandomSearch for VotingClassifier

In [ ]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1, max_iter=200)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = ComplementNB()

In [6]:
vectorizer = TfidfVectorizer
model = LogisticRegression

param = {
'vect__max_features': [1000, 4500, 6000],
'model__random_state': [1],
'model__tol': [1e-4, 1e-5],
'model__C': [1.0, 3.0],
'model__max_iter' : [200, 300],
'model__multi_class' : ['multinomial']
}

pipe = Pipeline([
        ('vect', vectorizer()),
        ('model', model())
    ])

gs = RandomizedSearchCV(pipe,
                      param_distributions=param,  # what parameters values are we searching?
                      cv=10,
                      n_iter=30,
                      scoring='accuracy', verbose=10,
                      n_jobs=-1)  # 5-fold cross-validation.


# Fit GridSearch to training data.
gs.fit(train_set["tokenized_text_string"], y_train)

joblib.dump(gs, 'best_logreg.pkl')
print(accuracy_score(y_test, gs.predict(test_set["tokenized_text_string"])))

C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\model_selection\_search.py:282: UserWarning: The total space of parameters 24 is smaller than n_iter=30. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 25.6min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 41.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 48.4min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 57.4min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 68.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 78.2min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 90.4min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 103.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 117.8min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 133.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 149.2min
[Par

0.08023220287198289


In [7]:
vectorizer = TfidfVectorizer
model = RandomForestClassifier

param = {
'vect__max_features': [1000, 4500, 6000],
'model__random_state':  [1],
'model__n_estimators':  [50, 100],
'model__min_samples_split': [2,3,4]
}

pipe = Pipeline([
        ('vect', vectorizer()),
        ('model', model())
    ])

gs = RandomizedSearchCV(pipe,
                      param_distributions=param,  # what parameters values are we searching?
                      cv=10,
                      n_iter=30,
                      scoring='accuracy', verbose=10,
                      n_jobs=-1)  # 5-fold cross-validation.


# Fit GridSearch to training data.
gs.fit(train_set["tokenized_text_string"], y_train)

joblib.dump(gs, 'best_forest.pkl')
print(accuracy_score(y_test, gs.predict(test_set["tokenized_text_string"])))

Fitting 10 folds for each of 18 candidates, totalling 180 fits


C:\Users\Euge\Miniconda3\lib\site-packages\sklearn\model_selection\_search.py:282: UserWarning: The total space of parameters 18 is smaller than n_iter=30. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 29.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 45.9min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 61.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 88.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 103.8min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 117.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 132.1min
[Parallel(n_jobs=-1)]: Do

0.07381607088298198


In [9]:
vectorizer = TfidfVectorizer
model = ComplementNB


param = {
'model__alpha': [0.001, 0,1, 0.5, 1],
'model__norm': [0, 1]
}

pipe = Pipeline([
        ('model', model())
])

gs = GridSearchCV(pipe,
                      param_grid=param,  # what parameters values are we searching?
                      cv=10,
                      scoring='accuracy', verbose=10,
                      n_jobs=-1)  # 5-fold cross-validation.


# Fit GridSearch to training data.
gs.fit(X_train, y_train)

joblib.dump(gs, 'best_NB.pkl')
print(accuracy_score(y_test, gs.predict(X_test)))

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

# Count of distribution for train and test set

In [33]:
np.unique(y_train, return_counts=True)[1]

array([1391, 1517, 1625, 1349, 1513, 1530, 1614, 1504, 1467, 1560, 1553,
       1696], dtype=int64)

In [146]:
np.unique(y_test, return_counts=True)[1]/len(y_test)*100

array([ 8.79926673,  6.27253284, 17.90406355,  6.38557898,  7.30522456,
        8.26153376,  8.25542316,  6.47723801,  7.57714635,  4.55239841,
       11.73235564,  6.47723801])

In [147]:
np.unique(y_train, return_counts=True)[1]/len(y_train)*100

array([7.59320924, 8.28101971, 8.87057154, 7.36393908, 8.25918445,
       8.35198428, 8.81052459, 8.21005513, 8.00807904, 8.51574868,
       8.47753698, 9.25814728])

In [32]:
np.mean(np.unique(y_test, return_counts=True)[1]/len(y_test)*100)

8.333333333333334